In [3]:
import numpy as np
import pandas as pd

In [1]:
import matplotlib.pyplot as plt
import sklearn 

# Các lỗi với dữ liệu cần làm sạch 

## Dữ liệu bị thiếu 

In [4]:
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [25, np.nan, 30, 22, np.nan],
    'Gender': ['F', 'M', np.nan, 'M', 'F'],
    'Salary': [50000, 60000, 55000, np.nan, 52000]
}

df = pd.DataFrame(data)
print(df)

      Name   Age Gender   Salary
0    Alice  25.0      F  50000.0
1      Bob   NaN      M  60000.0
2  Charlie  30.0    NaN  55000.0
3    David  22.0      M      NaN
4      Eve   NaN      F  52000.0


In [7]:
# Kiểm tra các cột có dữ liệu bị thiếu trong bảng 
df2=df.isnull().sum()

In [8]:
type(df2)

pandas.core.series.Series

In [16]:
type(df.isnull().sum()>0)

pandas.core.series.Series

In [13]:
df.isnull().sum()[df.isnull().sum()>0]

Age       2
Gender    1
Salary    1
dtype: int64

In [17]:
df.isnull().sum()[df.isnull().sum()>0]

Age       2
Gender    1
Salary    1
dtype: int64

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# Viết các hàm để xử lý giá trị bị thiếu 
# Hàm số 1: Dữ liệu bị thiếu quá nhiều thì bỏ cột (dùng cho cả categories và num)
class DropHighMisssingColumns(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.5):
        self.threshold=threshold
        self.columns_to_drop=[]
    def fit(self, X, y=None):
        missing_ratio=X.isnull().mean()
        self.columns_to_drop=missing_ratio[missing_ratio>self.threshold].index.tolist()
        return self
    def transform(self, X, y=None):
        return X.drop(columns=self.columns_to_drop, errors='ignore')

In [19]:
class DropHighMissingColumns(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.5):
        self.threshold=threshold
        self.columns_to_drop=[]
    def fit(self, X, y=None):
        missing_ratio=X.isnull().mean()
        self.columns_to_drop=missing_ratio[missing_ratio>self.threshold].index.tolist()
        return self
    def transform(self, X, y=None):
        return X.drop(columns=self.columns_to_drop)


In [20]:
# Viết hàm số 2: Xử lý giá trị bị thiếu cho dữ liẹu categories, thay giá trị bị thiếu bằng mode 
class ImputeMissingMode(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns=columns
        self.fill_values={}
    def fit(self, X,y=None):
        for column in self.columns:
            self.fill_values[column]=X[column].mode().iloc[0]
        return self
    def transform(self, X, y=None):
        X2=X.copy()
        for column, values in self.fill_values.item():
            X2[column]=X2[column].fillna(values)
        return X2



In [21]:
# Viết hàm xử lý số 2, hàm bị thiếu cho dữ liệu dạng categories, thay bằng mode
class ImputeMissingByMode(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns=columns
        self.fill_values={}
    def fit(self, X, y=None):
        for column in self.columns:
            self.fill_values[column]=X[column].mode().iloc[0]
    def transform(self, X, y=None):
        X2=X.copy()
        for column, value in self.fill_values.item():
            X2[column]=X2[column].fillna(value)
        return X2


In [ ]:
# Viết hàm xử lý số 3, thay thế giá trị bị thiếu bằng mean/ median
class ImputeMissingByMean(BaseEstimator, TransformerMixin):
    def __init__(self, columns, mode):
        self.columns=columns
        self.fill_values={}
        self.mode=mode
    def fit(self, X,y=None):
        for column in self.columns:
            if self.mode=='mean':
                self.fill_values[column]=X[column].mean()
            if self.mode=='median':
                self.fill_values[column]=X[column].median()
    def transform(self, X, y=None):
        X2=X.copy()
        for column, value in self.fill_values.item():
            X2[column]=X2[column].fillna(value)
        return X2

# Loại bỏ Outlier trong dữ liệu

In [22]:
# Phát hiện Outlier trong dữ liệu 
class OutlierTransformByBound1(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns=columns
        self.bounds={}
    def fit(self, X, y=None):
        for column in self.columns: 
            Q1=X[column].quantitle(0.25)
            Q3=X[column].quantitle(0.75)
            IQR=Q3-Q1
            lower=Q1-1.5*IQR
            upper=Q3+1.5*IQR
            self.bounds[column]=(lower, upper)
        return self
    
    def transform(self, X, y=None):
        X2=X.copy()
        for column, (lower, upper) in self.bounds:
            X2[column]=np.where(X2[column]<lower, lower, X2[column]>upper, upper)
        return X2




In [23]:
class OutlierTransformByBound2(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns=columns
        self.bounds={}
    def fit(self, X,y=None):
        for column in self.columns:
            mean=X[column].mean()
            std=X[column].std()
            self.bounds[column]=(mean, std)
        return self
    def transform(self, X, y=None):
        X2=X.copy()
        for column, (mean, std) in self.bounds:
            X2[column]=np.where(X2[column]<mean-3*std, mean-3*std, X2[column]>mean+3*std, mean+3*std)
        return X2